In [1]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd

# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [3]:
# 인허가 정보 파일 확인
file_list = os.listdir(f'{rawdata_path}/인허가데이터')
file_list[:5]

['6110000_서울특별시_03_10_01_P_비디오물감상실업.csv',
 '6110000_서울특별시_03_10_02_P_비디오물배급업.csv',
 '6110000_서울특별시_03_10_03_P_비디오물소극장업.csv',
 '6110000_서울특별시_03_10_04_P_비디오물시청제공업.csv',
 '6110000_서울특별시_03_10_05_P_비디오물제작업.csv']

In [4]:
# re.sub로 한글만 남기려 했는데 왠지 모르게 적용이 안되서
# 다른 방법을 이용

type_list = []

for csv in file_list:
    type_name = csv[len('6110000_서울특별시_03_10_01_P_'):-4]
    type_list.append(type_name)

type_list[:5]

['비디오물감상실업',
 '비디오물배급업',
 '비디오물소극장업',
 '비디오물시청제공업',
 '비디오물제작업']

In [5]:
for i in range(10):
    print(type_list[10*i+ 0:10*i+ 10 ])

['비디오물감상실업', '비디오물배급업', '비디오물소극장업', '비디오물시청제공업', '비디오물제작업', '관광펜션업', '관광숙박업', '외국인관광도시민박업', '숙박업', '자동차야영장업']
['한옥체험업', '국내여행업', '종합여행업', '영화배급업', '영화상영업', '영화상영관', '영화수입업', '영화제작업', '음반.음악영상물배급업', '온라인음악서비스제공업']
['음반.음악영상물제작업', '음반물배급업', '음반물제작업', '옥외광고업', '인쇄사', '출판사', '미용업', '이용업', '세탁업', '의료기관세탁물처리업']
['위탁급식영업', '집단급식소식품판매업', '집단급식소', '건강기능식품유통전문판매업', '건강기능식품일반판매업', '식품냉동냉장업', '축산판매업', '식육포장처리업', '식품소분업', '식품운반업']
['식품자동판매기업', '옹기류제조업', '식품판매업기타', '식품제조가공업', '식품첨가물제조업', '용기·포장지제조업', '유ᄐ

In [6]:
## 상권 분석에 필요한 업종 선정
store_type = ['영화상영관', '미용업', '이용업','세탁업', '일반음식점', '휴게음식점', '제과점영업', '동물병원', '동물미용업', '병원', '의원','약국']

In [7]:
selected_csv = [csv for csv in file_list
                    if any(store_type_item in csv
                         for store_type_item in store_type)]
selected_csv

['6110000_서울특별시_03_13_02_P_영화상영관.csv',
 '6110000_서울특별시_05_18_01_P_미용업.csv',
 '6110000_서울특별시_05_19_01_P_이용업.csv',
 '6110000_서울특별시_06_20_01_P_세탁업.csv',
 '6110000_서울특별시_07_24_04_P_일반음식점.csv',
 '6110000_서울특별시_07_24_05_P_휴게음식점.csv',
 '6110000_서울특별시_07_22_18_P_제과점영업.csv',
 '6110000_서울특별시_02_03_11_P_동물미용업.csv',
 '6110000_서울특별시_01_01_01_P_병원.csv',
 '6110000_서울특별시_01_01_02_P_의원.csv',
 '6110000_서울특별시_01_01_06_P_약국.csv',
 '6110000_서울특별시_02_03_01_P_동물병원.csv',
 '6110000_서울특별시_02_03_02_P_동물약국.csv']

In [8]:
column_list = ['번호','개방서비스명','인허가일자', '인허가취소일자', '영업상태구분코드', '영업상태명', '상세영업상태코드', '폐업일자',
               '소재지전체주소', '도로명전체주소', '사업장명', '업태구분명', '업종']

df = pd.DataFrame(columns = column_list)

for file_name in selected_csv:

    folder_path = f'{rawdata_path}/인허가데이터/'
    path = folder_path + file_name

    temp = pd.read_csv(path, encoding = 'cp949', low_memory = False)

    # 업태구분명이 -> 중분류
    # 업종 -> 소분류
    temp['업태구분명'] = temp.apply(lambda x: x['개방서비스명'] if pd.isna(x['업태구분명']) else x['업태구분명'], axis = 1)
    temp['업종'] = temp.apply(lambda x: x['개방서비스명'] + ' ' + x['업태구분명'], axis = 1)

    # 모든 데이터를 같은 양식으로 바꾸기 위한 준비
    temp = temp[column_list]

    ### 소재지 주소에서 시군구/읍면동 추출 - 모든 데이터는 서울시 데이터라 시도 구분은 필요 없음
    temp['시군구'] = temp['소재지전체주소'].apply(lambda x: x.split()[1]  if isinstance(x, str) and len(x.split()) > 1 else None)
    temp['읍면동']  = temp['소재지전체주소'].apply(lambda x: x.split()[2]  if isinstance(x, str) and len(x.split()) > 2 else None)

    ###  인허가 / 폐업일자 확인 -> 연도별 영업 상태 확인을 위해서
    #-> 다른 자료들이 시계열 자료가 아니므로 이번 프로젝트에서 미사용

    temp['인허가일자'] = pd.to_datetime(temp['인허가일자'], errors='coerce')

    # 폐업일자가 없는 경우 현재 영업중으로 간주
    # 폐업일자 nan -> 2024-04-15로
    temp['폐업일자'] = temp['폐업일자'].apply(lambda x: '2024-04-15' if pd.isna(x) else x)
    temp['폐업일자'] = pd.to_datetime(temp['폐업일자'], errors='coerce')

    # 연도별 해당 업체 운영 여부 확인
    # 영업기준 - 해당 년도 3월31일에 영업중인 곳
    for i in range(2016, 2025):
        temp_var1 = 'run' + str(i)
        open_date = str(i) + '-01-30'
        close_date = str(i) + '-01-31'
        temp[temp_var1] = temp.apply(lambda x: 1 if ((x['인허가일자'] <= pd.Timestamp(open_date)) &
                                                     (x['폐업일자'] > pd.Timestamp(close_date)))
                                                else 0,
                                                axis=1)

    run_col = temp.columns[-9:]
    temp['selected'] = temp[run_col].sum(axis=1)
    temp = temp[temp['selected'] > 0]
    temp = temp.drop(columns='selected')

    df = pd.concat([df, temp], axis = 0)

In [9]:
### 도로명 주소의 읍면동은 법정동 기준 -> 행정동 기준으로 바꿔주자

In [10]:
dong_convert = pd.read_csv(f'{data_path}/region_code_all.csv')
dong_convert.head()

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동


In [11]:
dong_convert['key'] = dong_convert['시군구'] + ' ' + dong_convert['법정동1']
dong_convert.head()

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2,key
0,11010720.0,1111051500,1111010100,종로구,청운효자동,청운효자동,청운동,청운동,종로구 청운효자동,종로구 청운효자동,종로구 청운동
1,11010720.0,1111051500,1111010200,종로구,청운효자동,청운효자동,신교동,신교동,종로구 청운효자동,종로구 청운효자동,종로구 신교동
2,11010720.0,1111051500,1111010300,종로구,청운효자동,청운효자동,궁정동,궁정동,종로구 청운효자동,종로구 청운효자동,종로구 궁정동
3,11010720.0,1111051500,1111010400,종로구,청운효자동,청운효자동,효자동,효자동,종로구 청운효자동,종로구 청운효자동,종로구 효자동
4,11010720.0,1111051500,1111010500,종로구,청운효자동,청운효자동,창성동,창성동,종로구 청운효자동,종로구 청운효자동,종로구 창성동


In [12]:
dong_convert[dong_convert['key'].duplicated()]

,ADM_CD,행정동코드,법정동코드,시군구,행정동1,행정동2,법정동1,법정동2,key1,key2,key
19,11010530.0,1111053000,1111011900,종로구,사직동,사직동,세종로,세종로,종로구 사직동,종로구 사직동,종로구 세종로
45,11010610.0,1111061500,1111011900,종로구,종로1234가동,종로1234가동,세종로,세종로,종로구 종로1234가동,종로구 종로1234가동,종로구 세종로
82,11010730.0,1111065000,1111013000,종로구,혜화동,혜화동,와룡동,와룡동,종로구 혜화동,종로구 혜화동,종로구 와룡동
89,11010680.0,1111068000,1111017400,종로구,창신2동,창신제2동,창신동,창신동,종로구 창신2동,종로구 창신제2동,종로구 창신동
90,11010690.0,1111069000,1111017400,종로구,창신3동,창신제3동,창신동,창신동,종로구 창신3동,종로구 창신제3동,종로구 창신동
...,...,...,...,...,...,...,...,...,...,...,...
733,11250730.0,1174061000,1174010900,강동구,천호2동,천호제2동,천호동,천호동,강동구 천호2동,강동구 천호제2동,강동구 천호동
734,11250630.0,1174062000,1174010900,강동구,천호3동,천호제3동,천호동,천호동,강동구 천호3동,강동구 천호제3동,강동구 천호동
736,11250660.0,1174065000,1174010800,강동구,성내2동,성내제2동,성내동,성내동,강동구 성내2동,강동구 성내제2동,강동구 성내동
737,11250670.0,1174066000,1174010800,강동구,성내3동,성내제3동,성내동,성내동,강동구 성내3동,강동구 성내제3동,강동구 성내동


In [13]:
df['key'] = df['시군구'] + ' ' + df['읍면동']
df.head()

,번호,개방서비스명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,폐업일자,소재지전체주소,도로명전체주소,...,run2016,run2017,run2018,run2019,run2020,run2021,run2022,run2023,run2024,key
6,7,영화상영관,2003-02-12,NaN,3,폐업,03,2024-04-15,"서울특별시 종로구 관수동 59-7번지 4층, 5중·상층",NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,종로구 관수동
7,8,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,"서울특별시 종로구 관수동 59-7 2층, 3중·상층","서울특별시 종로구 돈화문로 13, 2층, 3중·상층 (관수동)",...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,종로구 관수동
8,9,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,서울특별시 종로구 관수동 59-7 지하1층,"서울특별시 종로구 돈화문로 13, 지하1층 (관수동, 서울극장)",...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,종로구 관수동
9,10,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,서울특별시 종로구 관수동 59-7 5층,"서울특별시 종로구 돈화문로 13, 5층 (관수동)",...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,종로구 관수동
10,11,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,"서울특별시 종로구 관수동 59-7 1,2층","서울특별시 종로구 돈화문로 13, 1,2층 (관수동)",...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,종로구 관수동


In [14]:
df[df['key'].fillna('').str.contains('세종로')]


,번호,개방서비스명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,폐업일자,소재지전체주소,도로명전체주소,...,run2016,run2017,run2018,run2019,run2020,run2021,run2022,run2023,run2024,key
8736,8737,미용업,2018-04-26,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 202-1번지 401호,"서울특별시 종로구 세종대로 159, 4층 401호 (세종로)",...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,종로구 세종로
9588,9589,미용업,2014-08-18,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 77-6번지,"서울특별시 종로구 세종대로 209, 지1층 (세종로, 서울정부종합청사)",...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,종로구 세종로
26049,26050,미용업,2020-09-21,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 160-2,"서울특별시 종로구 세종대로23길 5, 3층 (세종로)",...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,종로구 세종로
2538,2539,이용업,2009-03-18,NaN,3,폐업,2,2018-07-02,서울특별시 종로구 세종로 77-6번지 B117호,"서울특별시 종로구 세종대로 209 (세종로,B117호)",...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,종로구 세종로
2893,2894,이용업,2018-09-13,NaN,3,폐업,2,2023-06-02,서울특별시 종로구 세종로 77-6 정부서울청사 본관 지하1층,"서울특별시 종로구 세종대로 209, 정부서울청사 본관 지하1층 (세종로)",...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,종로구 세종로
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,549,의원,2020-09-16,NaN,1,영업/정상,13,2024-04-15,서울특별시 종로구 세종로 202-1,"서울특별시 종로구 세종대로 159, 5층 (세종로)",...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,종로구 세종로
682,683,의원,2008-05-30,NaN,1,영업/정상,13,2024-04-15,서울특별시 종로구 세종로 202번지 1호 6층,"서울특별시 종로구 세종대로 159, 6층 (세종로)",...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,종로구 세종로
814,815,의원,2007-12-12,NaN,1,영업/정상,13,2024-04-15,서울특별시 종로구 세종로 202번지 1호 세종로202-1,"서울특별시 종로구 세종대로 159-0 (세종로, 세광빌딩 8층)",...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,종로구 세종로
1,2,약국,2015-02-16,2016-03-09,3,폐업,3,2016-03-09,서울특별시 종로구 세종로 77번지 6호 정부서울청사 1층,"서울특별시 종로구 세종대로 209, 1층 (세종로, 정부서울청사)",...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,종로구 세종로


In [15]:
df_merged = df.merge(dong_convert,
                     how = 'left',
                     on = 'key')

df_merged.head()

,번호,개방서비스명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,폐업일자,소재지전체주소,도로명전체주소,...,ADM_CD,행정동코드,법정동코드,시군구_y,행정동1,행정동2,법정동1,법정동2,key1,key2
0,7,영화상영관,2003-02-12,NaN,3,폐업,03,2024-04-15,"서울특별시 종로구 관수동 59-7번지 4층, 5중·상층",NaN,...,11010610.0,1.111062e+09,1.111016e+09,종로구,종로1234가동,종로1234가동,관수동,관수동,종로구 종로1234가동,종로구 종로1234가동
1,8,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,"서울특별시 종로구 관수동 59-7 2층, 3중·상층","서울특별시 종로구 돈화문로 13, 2층, 3중·상층 (관수동)",...,11010610.0,1.111062e+09,1.111016e+09,종로구,종로1234가동,종로1234가동,관수동,관수동,종로구 종로1234가동,종로구 종로1234가동
2,9,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,서울특별시 종로구 관수동 59-7 지하1층,"서울특별시 종로구 돈화문로 13, 지하1층 (관수동, 서울극장)",...,11010610.0,1.111062e+09,1.111016e+09,종로구,종로1234가동,종로1234가동,관수동,관수동,종로구 종로1234가동,종로구 종로1234가동
3,10,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,서울특별시 종로구 관수동 59-7 5층,"서울특별시 종로구 돈화문로 13, 5층 (관수동)",...,11010610.0,1.111062e+09,1.111016e+09,종로구,종로1234가동,종로1234가동,관수동,관수동,종로구 종로1234가동,종로구 종로1234가동
4,11,영화상영관,2003-02-12,NaN,3,폐업,03,2021-09-07,"서울특별시 종로구 관수동 59-7 1,2층","서울특별시 종로구 돈화문로 13, 1,2층 (관수동)",...,11010610.0,1.111062e+09,1.111016e+09,종로구,종로1234가동,종로1234가동,관수동,관수동,종로구 종로1234가동,종로구 종로1234가동


In [16]:
df_merged[df_merged['key'].fillna('').str.contains('세종로')]


,번호,개방서비스명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,폐업일자,소재지전체주소,도로명전체주소,...,ADM_CD,행정동코드,법정동코드,시군구_y,행정동1,행정동2,법정동1,법정동2,key1,key2
28211,8737,미용업,2018-04-26,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 202-1번지 401호,"서울특별시 종로구 세종대로 159, 4층 401호 (세종로)",...,11010720.0,1.111052e+09,1.111012e+09,종로구,청운효자동,청운효자동,세종로,세종로,종로구 청운효자동,종로구 청운효자동
28212,8737,미용업,2018-04-26,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 202-1번지 401호,"서울특별시 종로구 세종대로 159, 4층 401호 (세종로)",...,11010530.0,1.111053e+09,1.111012e+09,종로구,사직동,사직동,세종로,세종로,종로구 사직동,종로구 사직동
28213,8737,미용업,2018-04-26,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 202-1번지 401호,"서울특별시 종로구 세종대로 159, 4층 401호 (세종로)",...,11010610.0,1.111062e+09,1.111012e+09,종로구,종로1234가동,종로1234가동,세종로,세종로,종로구 종로1234가동,종로구 종로1234가동
30757,9589,미용업,2014-08-18,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 77-6번지,"서울특별시 종로구 세종대로 209, 지1층 (세종로, 서울정부종합청사)",...,11010720.0,1.111052e+09,1.111012e+09,종로구,청운효자동,청운효자동,세종로,세종로,종로구 청운효자동,종로구 청운효자동
30758,9589,미용업,2014-08-18,NaN,1,영업/정상,1,2024-04-15,서울특별시 종로구 세종로 77-6번지,"서울특별시 종로구 세종대로 209, 지1층 (세종로, 서울정부종합청사)",...,11010530.0,1.111053e+09,1.111012e+09,종로구,사직동,사직동,세종로,세종로,종로구 사직동,종로구 사직동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1155418,2,약국,2015-02-16,2016-03-09,3,폐업,3,2016-03-09,서울특별시 종로구 세종로 77번지 6호 정부서울청사 1층,"서울특별시 종로구 세종대로 209, 1층 (세종로, 정부서울청사)",...,11010530.0,1.111053e+09,1.111012e+09,종로구,사직동,사직동,세종로,세종로,종로구 사직동,종로구 사직동
1155419,2,약국,2015-02-16,2016-03-09,3,폐업,3,2016-03-09,서울특별시 종로구 세종로 77번지 6호 정부서울청사 1층,"서울특별시 종로구 세종대로 209, 1층 (세종로, 정부서울청사)",...,11010610.0,1.111062e+09,1.111012e+09,종로구,종로1234가동,종로1234가동,세종로,세종로,종로구 종로1234가동,종로구 종로1234가동
1155560,277,약국,2017-02-07,NaN,1,영업/정상,13,2024-04-15,서울특별시 종로구 세종로 202번지 1호,"서울특별시 종로구 세종대로 159, 4층 (세종로)",...,11010720.0,1.111052e+09,1.111012e+09,종로구,청운효자동,청운효자동,세종로,세종로,종로구 청운효자동,종로구 청운효자동
1155561,277,약국,2017-02-07,NaN,1,영업/정상,13,2024-04-15,서울특별시 종로구 세종로 202번지 1호,"서울특별시 종로구 세종대로 159, 4층 (세종로)",...,11010530.0,1.111053e+09,1.111012e+09,종로구,사직동,사직동,세종로,세종로,종로구 사직동,종로구 사직동


In [17]:
df_merged['key1'].nunique()

426

In [18]:
store1 = pd.DataFrame()

for year in range(2016, 2025):
    temp = df_merged[df_merged[f'run{year}'] == 1]

    temp_store = temp.groupby(['ADM_CD','key1','업종'])[['번호']].count().reset_index()
    temp_store = temp_store.pivot(columns="업종",index = ['ADM_CD','key1'], values = '번호')
    temp_store.reset_index(inplace = True)
    temp_store.fillna(0, inplace = True)
    temp_store.columns = ['ADM_CD', 'key1'] + [col + f'_{year}' for col in temp_store.columns[2:]]

    if year == 2016:
        store1 = temp_store
    else :
        store1 = store1.merge(temp_store,
                              how = 'outer',
                              on = ['ADM_CD','key1'])

In [19]:
store1.isna().sum().sum()

0

In [20]:
store2 = pd.DataFrame()

for year in range(2016, 2025):
    temp = df_merged[df_merged[f'run{year}'] == 1]

    temp_store = temp.groupby(['ADM_CD','key1','개방서비스명'])[['번호']].count().reset_index()
    temp_store = temp_store.pivot(columns="개방서비스명",index = ['ADM_CD','key1'], values = '번호')
    temp_store.reset_index(inplace = True)
    temp_store.fillna(0, inplace = True)
    temp_store.columns = ['ADM_CD', 'key1'] + [col + f'_{year}' for col in temp_store.columns[2:]]

    if year == 2016:
        store2 = temp_store
    else :
        store2 = store2.merge(temp_store,
                              how = 'outer',
                              on = ['ADM_CD','key1'])

In [21]:
final = store1.merge(store2,
                     how = 'left',
                     on = ['ADM_CD', 'key1'])
final.head()

,ADM_CD,key1,동물미용업 동물미용업_2016,동물병원 동물병원_2016,동물약국 동물약국_2016,미용업 기타_2016,미용업 네일아트업_2016,미용업 메이크업업_2016,미용업 미용업 기타_2016,미용업 숙박업 기타_2016,...,미용업_2024,병원_2024,세탁업_2024,약국_2024,영화상영관_2024,의원_2024,이용업_2024,일반음식점_2024,제과점영업_2024,휴게음식점_2024
0,11010530.0,종로구 사직동,0.0,1.0,2.0,1.0,5.0,2.0,0.0,0.0,...,72.0,2.0,8.0,14.0,4.0,55.0,10.0,1088.0,20.0,223.0
1,11010540.0,종로구 삼청동,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,1.0,3.0,0.0,7.0,0.0,273.0,3.0,91.0
2,11010550.0,종로구 부암동,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12.0,0.0,5.0,1.0,0.0,4.0,1.0,141.0,7.0,65.0
3,11010560.0,종로구 평창동,0.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,...,24.0,1.0,8.0,5.0,0.0,9.0,2.0,134.0,4.0,75.0
4,11010570.0,종로구 무악동,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,11.0,1.0,3.0,4.0,0.0,3.0,0.0,50.0,3.0,20.0


In [22]:
final['ADM_CD'].nunique()


426

In [23]:
final.isna().sum().sum()

0

In [24]:
final.to_csv(f'{data_path}/final/store.csv', index = False)